#### Import Required Packages

In [19]:
from cloudvolume import CloudVolume
import numpy as np
import os
import random
import requests
import tifffile
import matplotlib.pyplot as plt
import fsspec
import zarr
import dask.array as da
import pandas as pd

#### FlyEM Hemibrain Dataset - Neuroglancer

In [3]:
volume = CloudVolume('precomputed://gs://neuroglancer-janelia-flyem-hemibrain/v1.0/segmentation', use_https=True, parallel=True)

In [4]:
print("FlyEM Hemibrain - Volume size (z, y, x):", volume.bounds.maxpt)

FlyEM Hemibrain - Volume size (z, y, x): [34432 39552 41408]


In [8]:
crop_shape = np.array([1000, 1000, 1000])
max_start = volume.bounds.maxpt - crop_shape
start = np.array([random.randint(0, max_start[0]),
                  random.randint(0, max_start[1]),
                  random.randint(0, max_start[2])])
end = start + crop_shape

In [6]:
img = volume[start[0]:end[0], start[1]:end[1], start[2]:end[2]]

Decompressing: 100%|██████████| 4624/4624 [00:00<00:00, 158448.90it/s]


In [7]:
img.shape

(1000, 1000, 1000, 1)

In [75]:
np.save("Data/flyem_hemibrain_crop.npy", img)

In [12]:
vol_keys = volume.info.keys()

In [ ]:
print("Types: ", volume.info['@type'])
print("Number of Channels: ", volume.info['num_channels'])
print("scale:", volume.info['scales'][0])
print("Data type:", volume.info['data_type'])
print("Segment Properties:", volume.info['segment_properties'])
print("Type:", volume.info['type'])
print("Segment Name Map:", volume.info['segment_name_map'])
print("Mesh:", volume.info['mesh'])


Types:  neuroglancer_multiscale_volume
Number of Channels:  1
scale: {'chunk_sizes': [[64, 64, 64]], 'compressed_segmentation_block_size': [8, 8, 8], 'encoding': 'compressed_segmentation', 'key': '2048.0x2048.0x2048.0', 'resolution': [2048, 2048, 2048], 'sharding': {'@type': 'neuroglancer_uint64_sharded_v1', 'data_encoding': 'gzip', 'hash': 'identity', 'minishard_bits': 0, 'minishard_index_encoding': 'gzip', 'preshift_bits': 6, 'shard_bits': 0}, 'size': [135, 155, 162]}
Data type: uint64
Segment Properties: segment_properties
Type: segmentation
Segment Name Map: names
Mesh: mesh


In [39]:
ng_meta_keys = [k for k in vol_keys if k!= 'scales']
ng_keys_rows = []
for key in ng_meta_keys:
    ng_keys_rows.append({'Field': key, 'Value': volume.info[key]})
ng_meta_keys_df = pd.DataFrame(ng_keys_rows)
ng_meta_keys_df.to_csv('flyem_hemibrain_ng_meta_keys.csv', index=False)


In [41]:
print(ng_meta_keys_df.to_markdown(index=False))

| Field              | Value                          |
|:-------------------|:-------------------------------|
| @type              | neuroglancer_multiscale_volume |
| data_type          | uint64                         |
| num_channels       | 1                              |
| type               | segmentation                   |
| mesh               | mesh                           |
| segment_name_map   | names                          |
| segment_properties | segment_properties             |


In [71]:
ng_meta_scales_rows = []
for (idx, scale) in enumerate(volume.info['scales']):
    row = {'Scale': idx}
    for key, value in scale.items():
        if key != 'sharding':
            row[key] = value
    ng_meta_scales_rows.append(row)
ng_meta_scales_df = pd.DataFrame(ng_meta_scales_rows)
ng_meta_scales_df.to_csv('flyem_hemibrain_ng_meta_scales.csv', index=False)

In [72]:
print(ng_meta_scales_df.to_markdown(index=False))

|   Scale | chunk_sizes    | compressed_segmentation_block_size   | encoding                | key                  | resolution         | size                  |
|--------:|:---------------|:-------------------------------------|:------------------------|:---------------------|:-------------------|:----------------------|
|       0 | [[64, 64, 64]] | [8, 8, 8]                            | compressed_segmentation | 8.0x8.0x8.0          | [8, 8, 8]          | [34432, 39552, 41408] |
|       1 | [[64, 64, 64]] | [8, 8, 8]                            | compressed_segmentation | 16.0x16.0x16.0       | [16, 16, 16]       | [17216, 19776, 20704] |
|       2 | [[64, 64, 64]] | [8, 8, 8]                            | compressed_segmentation | 32.0x32.0x32.0       | [32, 32, 32]       | [8608, 9888, 10352]   |
|       3 | [[64, 64, 64]] | [8, 8, 8]                            | compressed_segmentation | 64.0x64.0x64.0       | [64, 64, 64]       | [4304, 4944, 5176]    |
|       4 | [[64, 64, 64]] |

In [73]:
ng_meta_sharding_rows = []
for (idx, scale) in enumerate(volume.info['scales']):
    row = {'Scale': idx}
    if 'sharding' in scale:
        for key, value in scale['sharding'].items():
            row[key] = value
        ng_meta_sharding_rows.append(row)
ng_meta_sharding_df = pd.DataFrame(ng_meta_sharding_rows)
ng_meta_sharding_df.to_csv('flyem_hemibrain_ng_meta_sharding.csv', index=False)

In [74]:
print(ng_meta_sharding_df.to_markdown(index=False))

|   Scale | @type                          | data_encoding   | hash     |   minishard_bits | minishard_index_encoding   |   preshift_bits |   shard_bits |
|--------:|:-------------------------------|:----------------|:---------|-----------------:|:---------------------------|----------------:|-------------:|
|       0 | neuroglancer_uint64_sharded_v1 | gzip            | identity |                6 | gzip                       |               9 |           15 |
|       1 | neuroglancer_uint64_sharded_v1 | gzip            | identity |                6 | gzip                       |               9 |           12 |
|       2 | neuroglancer_uint64_sharded_v1 | gzip            | identity |                6 | gzip                       |               9 |            9 |
|       3 | neuroglancer_uint64_sharded_v1 | gzip            | identity |                6 | gzip                       |               9 |            6 |
|       4 | neuroglancer_uint64_sharded_v1 | gzip            | identit